# FORDATIS - Repositorium für Forschungsdaten der FhG

<table table border="1" align="left">
 <tbody>
    <tr>
      <td>Autor: </td>
      <td style="text-align: left">Antje Schroeder (antje.schroeder@zv.fraunhofer.de)</td>
    </tr>
    <tr>
      <td>Stand: </td>
      <td style="text-align: left">05/05/2023</td>
    </tr>
    <tr>
      <td>Zustand: </td>
      <td style="text-align: left">lauffähig, fertig</td>
    </tr>
 </tbody>
</table>

<br><br><br><br><br>

Voraussetzung: Das Skript **fordatis_get_items.ipynb** erzeugt die Eingabedatei fordatis_uuids.csv

Dieses Skript benutzt eine Eingabedatei, die die UUID (unique identifer) eines FORDATIS-Items enthält. Mit der UUID wird eine Abfrage auf die FORDATIS bistreams durch und gibt Metadaten der Bitstreams zurück. Diese werden in eine Datei geschrieben. <br>

* Eingabedatei: fordatis_item_uuids.csv</br>
* Ausgabedatei: fordatis_bitstream_metadata.csv</br>
* Datenverzeichnis: ./data</br>

Dieses Skript benutzt die DSpace 6 Restfull-API:
* https://fordatis.fraunhofer.de/apidocu.jsp
* https://wiki.lyrasis.org/display/DSDOC6x/REST+API

In [9]:
# Import modules

import urllib.request
import json
import pandas as pd
import csv
import datetime

In [10]:
#Define environment 

output_dir = "./data/"
input_uuids = output_dir + "fordatis_item_uuids.csv"
output_bitstream_metadata = output_dir + "fordatis_bitstream_metadata.csv"

base_url = "https://fordatis.fraunhofer.de/rest/items/"
bitstreams = "/bitstreams"

fields = ['item_uuid','bitstream_uuid', 'sequenceID', 'file_name', 'bundle_name', 'datei_format', 'mime_type', 'file_size', 'link']

i = 0

In [11]:
# Open output file
with open(output_bitstream_metadata, 'w', newline='', encoding="utf-8") as f: # prepare output file 
    write = csv.writer(f)
    write.writerow(fields) # write header to outputfile

In [12]:
# Open input file
input_file = open(input_uuids)

# Construct request url for each line in input file
for line in input_file:
    item_uuid = line.rstrip('\n')
    url = base_url + item_uuid + bitstreams
            
    # Read data and create a pandas dataframe
    dataset = urllib.request.urlopen(url).read()
    dataset = json.loads(dataset)
    df = pd.DataFrame(dataset)
    
    # Get no of records to iter
    no_of_records = len(df)
    
    # Fetch data from data frame 

    for i in range(no_of_records):
        bitstream_uuid = dataset[i]["uuid"]
        sequenceID = dataset[i]["sequenceId"]
        name = dataset[i]["name"]
        bundleName = dataset[i]["bundleName"]
        datei_format = dataset[i]["format"]
        datei_format = datei_format.replace("; ", " ")
        mimeType = dataset[i]["mimeType"]
        size = dataset[i]["sizeBytes"]
        link = dataset[i]["retrieveLink"]
        mylist = [[item_uuid, bitstream_uuid, sequenceID, name, bundleName, datei_format, mimeType, size, link]]
        
        with open(output_bitstream_metadata, 'a', newline='', encoding="utf-8") as f:
                write = csv.writer(f)
                write.writerows(mylist)
                
print("all things done.")
input_file.close()

all things done.
